# GPT-LLAMA-Q&A-Agent

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [1]:
# imports
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
!ollama pull llama3.2

python(35151) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [3]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [4]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [7]:
openai = OpenAI()

In [5]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [9]:
# Get gpt-4o-mini to answer, with streaming
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": question}],
    stream=True
)

print("Answer from GPT-4o-mini:\n")
for chunk in response:
    delta = chunk.choices[0].delta
    if hasattr(delta, "content"):
        print(delta.content, end="", flush=True)


Answer from GPT-4o-mini:

This line of code uses Python's generator expression to yield values from a set comprehension, specifically focusing on extracting the "author" key from a collection of `book` dictionaries. Let's break it down step by step:

1. **Set Comprehension**: 
   The portion within the braces `{ ... }` is a set comprehension, which creates a set of unique values. It iterates over the `books` collection and collects authors from each book. 

   ```python
   {book.get("author") for book in books if book.get("author")}
   ```

   - `for book in books`: This part iterates through each item in the `books` collection (which is assumed to be a list of dictionaries).
   - `book.get("author")`: Fetches the value associated with the "author" key from the `book` dictionary.
   - `if book.get("author")`: This condition filters out any books where the "author" key is either missing or evaluates to a falsy value (e.g., `None`, an empty string).

   As a result, the set comprehension

In [10]:
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}

In [15]:
payload = {
    "model": MODEL_LLAMA,
    "messages": [
        {"role": "user", "content": question}
    ],
    "stream": True
}

In [16]:
response = requests.post(OLLAMA_API, json=payload, headers=HEADERS, stream=True)

In [17]:
print("Answer from LLaMA 3.2:\n")
for line in response.iter_lines():
    if line:
        try:
            data = json.loads(line.decode("utf-8"))
            if "message" in data and "content" in data["message"]:
                print(data["message"]["content"], end="", flush=True)
        except json.JSONDecodeError:
            continue

Answer from LLaMA 3.2:

This line of code is using a combination of generator expressions, dictionary methods, and the `yield from` syntax.

Here's a breakdown:

- `{...}`: This creates an expression that iterates over something (in this case, `books`). The results are collected into a new iterable.

- `for book in books`: This part iterates over each item (`book`) in the `books` collection.

- `.get("author")`: For each item in the collection, this method retrieves the value associated with the key `"author"` from the dictionary-like object. If the key is not present or if the value cannot be converted to a string (for example, if the value is `None`), this method will return `None`.

- `{...}`: Another iterable expression that collects the results of `.get("author")` for each item in the collection.

- `yield from {...}`: This syntax tells Python to use the iterable expression as an iterable and yields its values one at a time. It's similar to how `yield*` works, but it requires the 